In [ ]:
import matplotlib
import numpy as np
import xarray as xr
import pandas as pd
import hvplot.xarray
from glob import glob
import proplot as plot
from wrf import getvar, to_np, latlon_coords, interplevel

import sys
sys.path.append('../scripts/')
from xin_cartopy import load_province

# Read lightning data

In [ ]:
flash_dir = '../data/lightning/subset/'
f_ltng = flash_dir + 'LtgFlashPortions20190725.csv'
ltng = pd.read_csv(f_ltng)

# set tropomi passing time (output from plot_ir_swath_traj.py)
pass_t = pd.to_datetime('2019-07-25 05:20')
begin_t = -180  # minutes before the TROPOMI passes
end_t = 0

delta = pd.to_datetime(ltng['timestamp']) - pass_t
ltng['delta'] = delta.dt.total_seconds()/60
subset = (begin_t < ltng['delta'])& (ltng['delta'] < end_t)
ltng = ltng[subset]

# Read wrfout* data

In [ ]:
# ds_wrf = xr.open_dataset(glob('../data/wrfchem/wrfout/20190725/lnox/wrfout_d03_2019-07-25_04:40:00*')[0])

# # Extract the pressure, geopotential height, and wind variables
# # p = getvar(ds_wrf._file_obj.ds, "pressure")
# # ua = getvar(ds_wrf._file_obj.ds, "ua", units="kt")
# # va = getvar(ds_wrf._file_obj.ds, "va", units="kt")
# p = getvar(ds_wrf._close, "pressure")
# ua = getvar(ds_wrf, "ua", units="kt")
# va = getvar(ds_wrf, "va", units="kt")

# # Get the lat/lon coordinates
# lats, lons = latlon_coords(p)

# # Interpolate u, and v winds to 200 hPa
# u_500 = interplevel(ua, p, 400)
# v_500 = interplevel(va, p, 400)

In [ ]:
# fig, axs = plot.subplots()
# step = 20
# axs.barbs(to_np(lons[::step,::step]), to_np(lats[::step,::step]),
#           to_np(u_500[::step, ::step]), to_np(v_500[::step, ::step]),
# #           transform=crs.PlateCarree(),
# #           length=6
#           )

# Read TROPOMI data

In [ ]:
ds = xr.open_dataset('../data/s5p_chem/tm5/S5P_TM5_L2__NO2____20190725T042743_20190725T060912_09219_01_020100_20190911T091424.nc', group='S5P')
ds_tm5 = xr.open_dataset('../data/s5p_chem/tm5/S5P_TM5_L2__NO2____20190725T042743_20190725T060912_09219_01_020100_20190911T091424.nc', group='TM5')
ds_lnox = xr.open_dataset('../data/s5p_chem/lnox_500/S5P_CHEM_L2__NO2____20190725T042743_20190725T060912_09219_01_020100_20190911T091424.nc', group='CHEM')
ds_lnox_low = xr.open_dataset('../data/s5p_chem/lnox_330/S5P_CHEM_L2__NO2____20190725T042743_20190725T060912_09219_01_020100_20190911T091424.nc', group='CHEM')
ds_lnox_high = xr.open_dataset('../data/s5p_chem/lnox_700/S5P_CHEM_L2__NO2____20190725T042743_20190725T060912_09219_01_020100_20190911T091424.nc', group='CHEM')
ds_nolnox = xr.open_dataset('../data/s5p_chem/nolnox/S5P_CHEM_L2__NO2____20190725T042743_20190725T060912_09219_01_020100_20190911T091424.nc', group='CHEM')

Pick good or fair pixels:

In [ ]:
ds_tm5 = ds_tm5.where(ds['no2_scd_flag']==0)
ds_lnox = ds_lnox.where(ds['no2_scd_flag']==0)
ds_lnox_low = ds_lnox_low.where(ds['no2_scd_flag']==0)
ds_lnox_high = ds_lnox_high.where(ds['no2_scd_flag']==0)
ds_nolnox = ds_nolnox.where(ds['no2_scd_flag']==0)

## Calculation of AMFs and VCDs

In [ ]:
amf = ds_lnox['scdClr']/ds_lnox['vcdGnd']*(1-ds['cloud_radiance_fraction_nitrogendioxide_window']) + ds_lnox['scdCld']/ds_lnox['vcdGnd']*ds['cloud_radiance_fraction_nitrogendioxide_window']
diff = amf - ds_lnox['amfTrop']

In [ ]:
amfTrop = ds_lnox['amfTrop']
vcdGnd_lnox = ds_lnox['vcdGnd']
vcdLno2 = ds_lnox['vcdGnd'] - ds_nolnox['vcdGnd']
vcdLno = ds_lnox['vcdGnd_no'] - ds_nolnox['vcdGnd_no']

In [ ]:
amf_lno2 = amfTrop * vcdGnd_lnox / vcdLno2
amf_lnox = amfTrop * vcdGnd_lnox / (vcdLno2 + vcdLno)

# this method below could lead to more missing data
# scdTrop = ds['nitrogendioxide_tropospheric_column'] * ds['air_mass_factor_troposphere']
# try this one
scdTrop = ds['nitrogendioxide_slant_column_density'] - ds['nitrogendioxide_stratospheric_column'] * ds['air_mass_factor_stratosphere']

lno2Trop = scdTrop/amf_lno2
lnoxTrop = scdTrop/amf_lnox

# Quickview of LNO$_2$ and LNO$_x$

In [ ]:
xlim=(245, 305); ylim=(190, 235)
lnoxTrop.plot(xlim=xlim, ylim=ylim, vmin=0, vmax=0.002)

In [ ]:
lno2Trop.plot(xlim=xlim, ylim=ylim, vmin=0, vmax=0.002)

# Comparisons of different NO$_2$Trop products

In [ ]:
fig, axs = plot.subplots(axwidth=4, proj='pcarree',
                         ncols=2, nrows=2, sharey=3)

extend = [118.2, 119.7, 31.5, 32.8]
lon_d = 0.5; lat_d = 0.5
vmin = -1.5; vmax=1.5
cmap = 'Balance' # vik

# --- subplot 0 ---
ax = axs[0]
m = ax.pcolormesh(ds['assembled_lon_bounds'],
                   ds['assembled_lat_bounds'],
                   6.02214e3*(ds_lnox['no2Trop'] - ds_tm5['no2Trop']),
                   vmin=vmin,
                   vmax=vmax,
                   levels=256,
                   cmap=cmap)
ax.format(title='500 mol/flash vs. TM5-MP')

# --- subplot 1 ---
ax = axs[1]
m = ax.pcolormesh(ds['assembled_lon_bounds'],
                   ds['assembled_lat_bounds'],
                   6.02214e3*(ds_nolnox['no2Trop'] - ds_tm5['no2Trop']),
                   vmin=vmin,
                   vmax=vmax,
                   levels=256,
                   cmap=cmap)
ax.format(title='0 mol/flash vs. TM5-MP')

# --- subplot 2 ---
ax = axs[2]
m = ax.pcolormesh(ds['assembled_lon_bounds'],
                   ds['assembled_lat_bounds'],
                   6.02214e3*(ds_lnox['no2Trop'] - ds_nolnox['no2Trop']),
                   vmin=vmin,
                   vmax=vmax,
                   levels=256,
                   cmap=cmap)
ax.format(title='500 mol/flash vs. 0 mol/flash')

# --- lightning scatter ---
s = ax.scatter(ltng['longitude'], ltng['latitude'],
               c=ltng['delta'],
               cmap='plasma_r',
               levels=6,
               s=2)
s.set_facecolor('none')

# --- subplot 3 ---
ax = axs[3]
crf_bounds = np.array([0, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95, 0.97, 1.0])
norm = matplotlib.colors.BoundaryNorm(boundaries=crf_bounds, ncolors=256)
m_crf = ax.pcolormesh(ds['assembled_lon_bounds'],
                   ds['assembled_lat_bounds'],
                   ds['cloud_radiance_fraction_nitrogendioxide_window'],
                   cmap='Ice_r',
                   norm=norm)
ax.format(title='CRF')

# format
axs.format(abc=True,
           abcloc='ul',
           abcstyle='(a)',
           lonlines=lon_d,
           latlines=lat_d,
           lonlim=(extend[0], extend[1]),
           latlim=(extend[2], extend[3]),
           dms=False,
           suptitle='$\Delta$ VCD'
          )

axs[:, 0].format(latlabels=True)
axs[0, :].format(lonlabels=True)

# colorbar
axs[1].colorbar(m, loc='r',
             ticks=plot.arange(-1.5, 1.5, 0.5),
             label='NO$_2$ (10$^{16}$ molec. / cm$^2$)',
             shrink=0.87)

axs[3].colorbar(s, loc='r',
#              ticks=plot.arange(-180, 0, 30),
             label='Flash Time (mins)',
             shrink=0.87)

axs[3].colorbar(m_crf,
                loc='r',
                ticks=crf_bounds,
                label='CRF',
                shrink=0.87,
                spacing="proportional")

# fig.savefig('../figures/delta_vcd_2019.png')

## Picking the specific pixels

Pick the pixels manually:

In [ ]:
(ds_lnox['no2Trop'] - ds_nolnox['no2Trop']).assign_coords(y=range(ds_lnox['no2Trop'].sizes['y']),
                                                          x=range(ds_lnox['no2Trop'].sizes['x'])).hvplot(xlim=xlim, ylim=ylim)

In [ ]:
(ds_lnox['no2Trop'] - ds_nolnox['no2Trop']).isel(x=278, y=211)

In [ ]:
(ds_lnox['no2Trop'] - ds_nolnox['no2Trop']).isel(x=284, y=214)

(ds_lnox['no2Trop'] - ds_nolnox['no2Trop']).isel(x=286, y=214)

Pick these pixels:

Positive:
- x=277, y=211
- x=278, y=211


Negative:
- x=284, y=214
- x=285, y=214
- x=286, y=214

## Reproducing the results

In [ ]:
def integPr(no2, s5p_p, psfc, ptropo):
    layername = no2.dims[0]

    # constants
    R = 287.3
    T0 = 273.15
    g0 = 9.80665
    p0 = 1.01325e5


    subcolumn = 10 * R *T0 /(g0*p0) \
              * no2*1e6 \
              * abs(s5p_p.diff(s5p_p.dims[0])) * 2.6867e16 # DU to moleclues/cm2
        
    sub_layer = (s5p_p <= psfc) & (s5p_p > ptropo)

    vcd = subcolumn.where(sub_layer[:-1, ...].values).sum(layername)
#     print(sub_layer[:-1, ...].isel(x=284, y=214))
#     print(subcolumn.where(sub_layer[:-1, ...].values).isel(x=284, y=214))

#     print(subcolumn, sub_layer[:-1, ...], layername)

    return vcd

In [ ]:
topest_pressure = xr.DataArray(np.full((1, ds_lnox.sizes['y'], ds_lnox.sizes['x']), 0),
                               dims=['plevel', 'y', 'x'])

s5p_p = xr.concat([ds_lnox['plevels'], topest_pressure], dim='plevel')

Check whether we can reproduce the integration results:

In [ ]:
# Test vcdGnd
vcdGnd_test = integPr(ds_lnox['no2apriori'], s5p_p, ds['surface_pressure']/1e2, ds_lnox['tropopause_pressure'])

(ds_lnox['vcdGnd'] - vcdGnd_test).plot(xlim=xlim, ylim=ylim, vmin=-1, vmax=1)

In [ ]:
scdClr = integPr(ds_lnox['no2apriori'] * ds_lnox['swClr'],
                 s5p_p,
                 ds['surface_pressure']/1e2, ds_lnox['tropopause_pressure'])

scdCld = integPr(ds_lnox['no2apriori'] * ds_lnox['swCld'],
                 s5p_p,
                 ds['surface_pressure']/1e2, ds_lnox['tropopause_pressure'])

amfClr = scdClr / vcdGnd_lnox
(ds_lnox['scdClr'] - scdClr).plot(xlim=xlim, ylim=ylim)

The small difference can be neglected.

Actually this is caused by the interpolated clearSW and cloudySW in the original program.

In [ ]:
integPr(ds_lnox['no2apriori'], s5p_p, ds['surface_pressure']/1e2, ds_lnox['tropopause_pressure']).isel(x=284, y=214).values - ds_lnox['vcdGnd'].isel(x=284, y=214)

In [ ]:
integPr(ds_nolnox['no2apriori'], s5p_p, ds['surface_pressure']/1e2, ds_nolnox['tropopause_pressure']).isel(x=284, y=214).values - ds_nolnox['vcdGnd'].isel(x=284, y=214)

# Comparisons of a priori profiles

## Positive pixels

In [ ]:
# - x=277, y=211
# - x=278, y=211

sel_x = xr.DataArray([277, 278])
sel_y = 211

fig, axs= plot.subplots()

chem_lnox_positve = ds_lnox['no2apriori'].isel(x=sel_x, y=sel_y).mean(dim='dim_0')
chem_lnox_high_positve = ds_lnox_high['no2apriori'].isel(x=sel_x, y=sel_y).mean(dim='dim_0')
chem_nolnox_positve = ds_nolnox['no2apriori'].isel(x=sel_x, y=sel_y).mean(dim='dim_0')
tm5_positve = ds_tm5['no2apriori'].isel(x=sel_x, y=sel_y).mean(dim='dim_0')

plevels_positve = ds_lnox['plevels'].isel(x=sel_x, y=sel_y).mean(dim='dim_0')

l1 = axs.plot(chem_nolnox_positve*1e12, plevels_positve, label='0 mol/flash')
l2 = axs.plot(chem_lnox_positve*1e12, plevels_positve, label='500 mol/flash')
l3 = axs.plot(chem_lnox_high_positve*1e12, plevels_positve, label='700 mol/flash')
l4 = axs.plot(tm5_positve*1e12, plevels_positve, label='TM5-MP')

l5 = axs.axhline(ds['cloud_pressure_crb'].isel(x=xr.DataArray([277, 278]), y=211).mean(dim='dim_0')/1e2,
                 linestyle='--',
                 label='Cloud Pressure',color='k')

axs.format(xlim=(0, 6e3), ylim=(1000, 100), xlabel='NO$_2$ (pptv)', ylabel='Pressure (hPa)')
fig.legend([l1, l2, l3, l4, l5], ncols=1)

## Negative pixels

In [ ]:
# - x=284, y=214
# - x=285, y=214
# - x=286, y=214

sel_x = xr.DataArray([284, 285, 286])
sel_y = 214

fig, axs= plot.subplots()

chem_lnox_negative = ds_lnox['no2apriori'].isel(x=sel_x, y=sel_y).mean(dim='dim_0')
chem_lnox_high_negative = ds_lnox['no2apriori'].isel(x=sel_x, y=sel_y).mean(dim='dim_0')
chem_nolnox_negative = ds_nolnox['no2apriori'].isel(x=sel_x, y=sel_y).mean(dim='dim_0')
tm5_negative = ds_tm5['no2apriori'].isel(x=sel_x, y=sel_y).mean(dim='dim_0')

plevels_negative = ds_lnox['plevels'].isel(x=sel_x, y=sel_y).mean(dim='dim_0')

l1 = axs.plot(chem_nolnox_negative*1e12, plevels_negative, label='0 mol/flash')
l2 = axs.plot(chem_lnox_negative*1e12, plevels_negative, label='500 mol/flash')
l3 = axs.plot(chem_lnox_high_negative*1e12, plevels_negative, label='700 mol/flash')
l4 = axs.plot(tm5_negative*1e12, plevels_negative, label='TM5-MP')

l5 = axs.axhline(ds['cloud_pressure_crb'].isel(x=xr.DataArray([277, 278]), y=211).mean(dim='dim_0')/1e2,
                 linestyle='--',
                 label='Cloud Pressure',color='k')

axs.format(xlim=(0, 6e3), ylim=(1000, 100), xlabel='NO$_2$ (pptv)', ylabel='Pressure (hPa)')
fig.legend([l1, l2, l3, l4, l5], ncols=1)

# Mean NO2 a priori profiles over convective pixels

All pixels:

In [ ]:
fig, axs = plot.subplots(axwidth=3)

means = (ds_tm5['no2apriori']*1e12).mean(dim=['x', 'y'])
stds = (ds_tm5['no2apriori']*1e12).std(dim=['x', 'y'])

l1 = axs.errorbar(means, plevels_positve, xerr=stds, label='TM5-MP')

means = (ds_lnox['no2apriori']*1e12).mean(dim=['x', 'y'])
stds = (ds_lnox['no2apriori']*1e12).std(dim=['x', 'y'])

l2 = axs.errorbar(means, plevels_positve, xerr=stds, label='500 mol/flash')

means = (ds_nolnox['no2apriori']*1e12).mean(dim=['x', 'y'])
stds = (ds_nolnox['no2apriori']*1e12).std(dim=['x', 'y'])

l3 = axs.errorbar(means, plevels_positve, xerr=stds, label='0 mol/flash')

fig.legend([l1, l2, l3], ncols=1)
axs.format(xlim=(-1, 4e3),
           ylim=(1000, 100),
           xlabel='NO$_2$ (pptv)',
           ylabel='Pressure (hPa)')

Convective pixels:

In [ ]:
crf = ds['cloud_radiance_fraction_nitrogendioxide_window']
subset = crf>= 0.7

fig, axs = plot.subplots(axwidth=3)

means = (ds_tm5['no2apriori'].where(subset)*1e12).mean(dim=['x', 'y'])
stds = (ds_tm5['no2apriori'].where(subset)*1e12).std(dim=['x', 'y'])

# l1 = axs.errorbar(means, plevels_positve, xerr=stds, label='TM5-MP')
l1 = axs.plot(means, plevels_positve, label='TM5-MP')

means = (ds_lnox_high['no2apriori'].where(subset)*1e12).mean(dim=['x', 'y'])
stds = (ds_lnox_high['no2apriori'].where(subset)*1e12).std(dim=['x', 'y'])

l2 = axs.plot(means, plevels_positve, label='700 mol/flash')

means = (ds_lnox['no2apriori'].where(subset)*1e12).mean(dim=['x', 'y'])
stds = (ds_lnox['no2apriori'].where(subset)*1e12).std(dim=['x', 'y'])

# l3 = axs.errorbar(means, plevels_positve, xerr=stds, label='300 mol/flash')
l3 = axs.plot(means, plevels_positve, label='500 mol/flash')


means = (ds_nolnox['no2apriori'].where(subset)*1e12).mean(dim=['x', 'y'])
stds = (ds_nolnox['no2apriori'].where(subset)*1e12).std(dim=['x', 'y'])

# l4 = axs.errorbar(means, plevels_positve, xerr=stds, label='0 mol/flash')
l4 = axs.plot(means, plevels_positve, label='0 mol/flash')


fig.legend([l1, l2, l3, l4], ncols=1)
axs.format(xlim=(-1e2, 2e3),
           ylim=(1000, 100),
           xlabel='NO$_2$ (pptv)',
           ylabel='Pressure (hPa)',
           yscale='log',
           ylocator=100,
           grid=False)

# Test of replacing profiles

**h1: p > 800 hPa**


**h2: 400 hPa <= p <= 800 hPa**

**h3: 150 hPa <= p <= 400 hPa**

~~850 hPa, 200 hPa~~

Note: The modified Ott profile has a bimodal distribution (4 km and 8 km)

- Surface to 800 hPa (2 km): No significant difference
- 800 hPa -- 400 hPa (7 km): first summit
- 400 hPa -- 150 hPa (13.5 km): second summit

## TM5 as the reference

**h1: p > 800 hPa**

    - TM5 v.s 0 mol/flash

**h2: 400 hPa <= p <= 800 hPa**

    - TM5 v.s 0 mol/flash
    - TM5 v.s 500 mol/flash

**h3: 150 hPa <= p <= 400 hPa**

    - TM5 v.s 0 mol/flash

In [ ]:
# def cal_amf(ds, ds_model, ds_model_apriori, s5p_p):
#     scdClr = integPr(ds_model_apriori * ds_model['swClr'],
#                      s5p_p,
#                      ds['surface_pressure']/1e2, ds_model['tropopause_pressure'])

#     scdCld = integPr(ds_model_apriori * ds_model['swCld'],
#                      s5p_p,
#                      ds['cloud_pressure_crb']/1e2, ds_model['tropopause_pressure'])

#     vcdGnd = integPr(ds_model_apriori, s5p_p, ds['surface_pressure']/1e2, ds_model['tropopause_pressure'])

#     amfClr = scdClr / vcdGnd
#     amfCld = scdCld / vcdGnd

#     crf = ds['cloud_radiance_fraction_nitrogendioxide_window']

#     amf = amfClr*(1-crf) + amfCld*crf

#     return amf

def cal_amf(ds, ds_lnox, da_mod, s5p_p):
    scdClr = integPr(da_mod * ds_lnox['swClr'],
                     s5p_p,
                     ds['surface_pressure']/1e2, ds_lnox['tropopause_pressure'])

    scdCld = integPr(da_mod * ds_nolnox['swCld'],
                     s5p_p,
                     ds['cloud_pressure_crb']/1e2, ds_lnox['tropopause_pressure'])

    vcdGnd = integPr(da_mod, s5p_p, ds['surface_pressure']/1e2, ds_lnox['tropopause_pressure'])

    amfClr = scdClr / vcdGnd
    amfCld = scdCld / vcdGnd

    crf = ds['cloud_radiance_fraction_nitrogendioxide_window']

    amf = amfClr*(1-crf) + amfCld*crf

    return amf

#### h1

In [ ]:
h1 = ds_nolnox['plevels'] > 800

ds_h1 = ds_nolnox['no2apriori'].where(~h1,  ds_tm5['no2apriori'])
amf_h1 = cal_amf(ds, ds_nolnox, ds_h1, s5p_p)

((amf_h1 - ds_nolnox['amfTrop'])/ds_nolnox['amfTrop']).plot(xlim=xlim, ylim=ylim)

#### h2

In [ ]:
h2 = (ds_nolnox['plevels'] >= 400) & (ds_nolnox['plevels'] <= 800)

ds_h2 = ds_nolnox['no2apriori'].where(~h2,  ds_tm5['no2apriori'])
amf_h2 = cal_amf(ds, ds_nolnox, ds_h2, s5p_p)

((amf_h2 - ds_nolnox['amfTrop'])/ds_nolnox['amfTrop']).plot(xlim=xlim, ylim=ylim)

In [ ]:
h2 = (ds_lnox['plevels'] >= 150) & (ds_lnox['plevels'] <= 400)

ds_h2 = ds_lnox['no2apriori'].where(~h2,  ds_tm5['no2apriori'])
amf_h2 = cal_amf(ds, ds_lnox, ds_h2, s5p_p)

((amf_h2 - ds_lnox['amfTrop'])/ds_lnox['amfTrop']).plot(xlim=xlim, ylim=ylim)

#### h3

In [ ]:
h3 = ds_nolnox['plevels'] < 150

ds_h3 = ds_nolnox['no2apriori'].where(~h3,  ds_tm5['no2apriori'])
amf_h3 = cal_amf(ds, ds_nolnox, ds_h3, s5p_p)

((amf_h3 - ds_nolnox['amfTrop'])/ds_nolnox['amfTrop']).plot(xlim=xlim, ylim=ylim)

As the difference is large and different over different regions, it's better to compare 0 mol/flash and 330 mol/flash:

## 0 mol/flash as the reference

0 mol/flash vs. 500 mol/flash:

**h1: p > 800 hPa**

**h2: 400 hPa <= p <= 800 hPa**

**h2: 150 hPa <= p <= 400 hPa**

#### h1

In [ ]:
h1 = ds_nolnox['plevels'] > 800

ds_h1 = ds_nolnox['no2apriori'].where(~h1,  ds_lnox['no2apriori'])
amf_h1 = cal_amf(ds, ds_nolnox, ds_h1, s5p_p)

((amf_h1 - ds_nolnox['amfTrop'])/ds_nolnox['amfTrop']).plot(xlim=xlim, ylim=ylim)

#### h2

In [ ]:
h2 = (ds_nolnox['plevels'] >= 400) & (ds_nolnox['plevels'] <= 800)

ds_h2 = ds_nolnox['no2apriori'].where(~h2,  ds_lnox['no2apriori'])
amf_h2 = cal_amf(ds, ds_nolnox, ds_h2, s5p_p)

((amf_h2 - ds_nolnox['amfTrop'])/ds_nolnox['amfTrop']).plot(xlim=xlim, ylim=ylim)

#### h3

In [ ]:
h3 = (ds_nolnox['plevels'] >= 150) & (ds_nolnox['plevels'] <= 400)

ds_h3 = ds_nolnox['no2apriori'].where(~h3,  ds_lnox['no2apriori'])
amf_h3 = cal_amf(ds, ds_nolnox, ds_h3, s5p_p)

((amf_h3 - ds_nolnox['amfTrop'])/ds_nolnox['amfTrop']).plot(xlim=xlim, ylim=ylim)

#### Overall

In [ ]:
amf_all = cal_amf(ds, ds_nolnox, ds_lnox['no2apriori'], s5p_p)
((amf_all - ds_nolnox['amfTrop'])/ds_nolnox['amfTrop']).plot(xlim=xlim, ylim=ylim)

## Comparisons of AMFs by replacing profiles

#### AMF$_{Trop}$

In [ ]:
def cal_amf(ds, ds_lnox, da_mod, s5p_p):
    scdClr = integPr(da_mod * ds_lnox['swClr'],
                     s5p_p,
                     ds['surface_pressure']/1e2, ds_lnox['tropopause_pressure'])

    scdCld = integPr(da_mod * ds_lnox['swCld'],
                     s5p_p,
                     ds['cloud_pressure_crb']/1e2, ds_lnox['tropopause_pressure'])

    vcdGnd = integPr(da_mod, s5p_p, ds['surface_pressure']/1e2, ds_lnox['tropopause_pressure'])

    amfClr = scdClr / vcdGnd
    amfCld = scdCld / vcdGnd

    crf = ds['cloud_radiance_fraction_nitrogendioxide_window']

    amf = amfClr*(1-crf) + amfCld*crf

    return amf

In [ ]:
def delta_amf(ds, ds_nolnox, ds_lnox, s5p_p, vmin, vmax, ax=None, p_min=None, p_max=None, cmap='RdBu_r', cut=0):
    '''
    Replace the a priori profile, recalculate AMF, and
        calculate the percent difference:
            (new_AMF - AMF_nolnox) / AMF_nolnox, units: %

    Finally, plot the percent difference on ax.
    
    Return the plot object.
    '''

    if p_min and p_max:
        p = (p_min <= ds_lnox['plevels']) & (ds_lnox['plevels'] <= p_max)
    elif p_min:
        p = ds_lnox['plevels'] < p_min
    elif p_max:
        p = ds_lnox['plevels'] > p_max
    else:
        p = xr.full_like(ds_lnox['plevels'], True, dtype='bool')

#     ds_new = ds_nolnox['no2apriori'].where(~p,  ds_replace['no2apriori'])
#     amf = cal_amf(ds, ds_nolnox, ds_new, s5p_p)
#     difference = (amf-ds_nolnox['amfTrop'])/ds_nolnox['amfTrop']*1e2

    da_mod = ds_nolnox['no2apriori'].where(~p, ds_lnox['no2apriori'])
    amf_mod = cal_amf(ds, ds_lnox, da_mod, s5p_p)
    amf = ds_nolnox['amfTrop']
    difference = (amf_mod-amf)/amf*1e2
    
#     da_mod = ds_lnox['no2apriori'].where(~p,  ds_nolnox['no2apriori'])
#     amf = cal_amf(ds, ds_lnox, da_mod, s5p_p)
# #     difference = (amf-ds_lnox['amfTrop'])/ds_lnox['amfTrop']*1e2
#     difference = (ds_lnox['amfTrop'] - amf)/amf*1e2

    if ax:
        m = ax.pcolormesh(ds['assembled_lon_bounds'],
                          ds['assembled_lat_bounds'],
                          difference,
                          vmin=vmin,
                          vmax=vmax,
                          levels=256,
                          cmap=cmap,
                          cmap_kw={'cut': cut})
    
        print('min_diff', difference.min(), 'max_diff', difference.max())
        return m

    else:
        return amf, amf_mod

    return m

In [ ]:
fig, axs = plot.subplots(axwidth=4, proj='pcarree',
                         ncols=4, nrows=2, sharey=3)

extend = [118.2, 119.7, 31.5, 32.8]
lon_d = 0.5; lat_d = 0.5
amftrop_min = -150; amftrop_max=150


## ---------------- AMF_Trop ---------------- ##
# --- subplot 0 ---
ax = axs[0]
m = delta_amf(ds, ds_nolnox, ds_lnox, s5p_p, amftrop_min, amftrop_max, ax=ax, p_max=800)
ax.format(title='Low layer (> 800 hPa)')

# --- lightning scatter ---
s = ax.scatter(ltng['longitude'], ltng['latitude'],
               c=ltng['delta'],
               cmap='plasma_r',
               levels=6,
               s=2)
s.set_facecolor('none')

ax.colorbar(s, loc='l',
            label='Flash Time (mins)',
            shrink=0.87)

# --- subplot 1 ---
ax = axs[1]
delta_amf(ds, ds_nolnox, ds_lnox, s5p_p, amftrop_min, amftrop_max, ax=ax, p_min=400, p_max=800)
ax.format(title='Middle layer (800 hPa - 400 hPa)')

# --- subplot 2 ---
ax = axs[2]
delta_amf(ds, ds_nolnox, ds_lnox, s5p_p, amftrop_min, amftrop_max, ax=ax, p_min=150, p_max=400)
ax.format(title='High layer (400 hPa - 150 hPa)')



# --- subplot 3 ---
ax = axs[3]
delta_amf(ds, ds_nolnox, ds_lnox, s5p_p, amftrop_min, amftrop_max, ax=ax)
ax.format(title='All layers')

# colorbar
ax.colorbar(m, loc='r',
            ticks=plot.arange(-150, 150, 50),
            minorticks=25,
            label='%$\Delta$ AMF$_{Trop}$ 500 vs. 0 mol/flash',
            shrink=0.87)

# format
axs.format(abc=True,
           abcloc='ul',
           abcstyle='(a)',
           lonlines=lon_d,
           latlines=lat_d,
           lonlim=(extend[0], extend[1]),
           latlim=(extend[2], extend[3]),
           dms=False,
#            suptitle='(modified AMF - AMF)/AMF'
          )

axs[:, 0].format(latlabels=True)
axs[1, :].format(lonlabels=True)



## ---------------- AMF_LNO2 ---------------- ##


# axs[3].colorbar(s, loc='r',
# #              ticks=plot.arange(-180, 0, 30),
#              label='Flash Time (mins)',
#              shrink=0.87)

# axs[3].colorbar(m_crf,
#                 loc='r',
#                 ticks=crf_bounds,
#                 label='CRF',
#                 shrink=0.87,
#                 spacing="proportional")

# fig.savefig('../figures/delta_vcd_2019.png')

## AMF$_{LNO_x}$

$ AMF_{LNO_x} = AMF_{Trop} * VCD_{Gnd} / VCD_{LNO_x} $

Replace the profile:

$ AMF_{LNO_x} = AMF_{Trop}(replace) * VCD_{Gnd}(replace) / VCD_{LNO_x}(replace) $

In [ ]:
(ds_lnox['amfTrop'] * ds_lnox['vcdGnd']).plot(xlim=xlim, ylim=ylim)

In [ ]:
(ds_lnox['vcdGnd']-ds_nolnox['vcdGnd'] + ds_lnox['vcdGnd_no']-ds_nolnox['vcdGnd_no']).plot(xlim=xlim, ylim=ylim)

In [ ]:
(ds_lnox['amfTrop'] * ds_lnox['vcdGnd'] / (ds_lnox['vcdGnd']-ds_nolnox['vcdGnd'] + ds_lnox['vcdGnd_no']-ds_nolnox['vcdGnd_no'])).assign_coords(y=range(ds_lnox['no2Trop'].sizes['y']),
                                                          x=range(ds_lnox['no2Trop'].sizes['x'])).hvplot(xlim=xlim, ylim=ylim, clim=(0,1), cmap='viridis')

Parameters causing the negative and large positive AMF$_{LNOx}$ values:

Pixels not affected by LNOx should have **small positive even negative** values.

So, it's better to clip the AMF_LNOx values to 0 ~ 1:

In [ ]:
def cal_amf_lnox(ds, ds_nolnox, ds_lnox, da_mod, da_mod_no, s5p_p):
    scdClr = integPr(da_mod * ds_lnox['swClr'],
                     s5p_p,
                     ds['surface_pressure']/1e2, ds_lnox['tropopause_pressure'])

    scdCld = integPr(da_mod * ds_lnox['swCld'],
                     s5p_p,
                     ds['cloud_pressure_crb']/1e2, ds_lnox['tropopause_pressure'])

    vcdGnd = integPr(da_mod, s5p_p, ds['surface_pressure']/1e2, ds_lnox['tropopause_pressure'])
    vcdGnd_no = integPr(da_mod_no, s5p_p, ds['surface_pressure']/1e2, ds_lnox['tropopause_pressure'])

    amfClr = scdClr / vcdGnd
    amfCld = scdCld / vcdGnd

    crf = ds['cloud_radiance_fraction_nitrogendioxide_window']

    amf = amfClr*(1-crf) + amfCld*crf
    
    amf_lnox = ds_lnox['amfTrop'] * ds_lnox['vcdGnd'] / (ds_lnox['vcdGnd']-ds_nolnox['vcdGnd'] + ds_lnox['vcdGnd_no']-ds_nolnox['vcdGnd_no'])
    amf_lnox_mod = amf * vcdGnd / (vcdGnd-ds_nolnox['vcdGnd'] + vcdGnd_no-ds_nolnox['vcdGnd_no'])

    return amf_lnox, amf_lnox_mod


def delta_amf_lnox(ds, ds_nolnox, ds_lnox, ds_lnox_high, s5p_p, vmin, vmax, ax=None, p_min=None, p_max=None, cmap='RdBu_r', cut=0):
    '''
    Replace the a priori profile, recalculate AMF, and
        calculate the percent difference:
            (new_AMF - AMF_nolnox) / AMF_nolnox, units: %

    Finally, plot the percent difference on ax.
    
    Return the plot object.
    '''

    if p_min and p_max:
        p = (p_min <= ds_lnox['plevels']) & (ds_lnox['plevels'] <= p_max)
    elif p_min:
        p = ds_lnox['plevels'] < p_min
    elif p_max:
        p = ds_lnox['plevels'] > p_max
    else:
        p = xr.full_like(ds_lnox['plevels'], True, dtype='bool')

    ds_mod = ds_lnox['no2apriori'].where(~p, ds_lnox_high['no2apriori'])
    ds_mod_no = ds_lnox['noapriori'].where(~p, ds_lnox_high['noapriori'])

    amf_lnox, amf_lnox_mod = cal_amf_lnox(ds, ds_nolnox, ds_lnox, ds_mod, ds_mod_no, s5p_p)

    # only plot the AMFs related to LNOx (0 < AMF < 10)
    valid = (0 < amf_lnox) & (amf_lnox <= 10)
    difference = (amf_lnox_mod-amf_lnox)/amf_lnox*1e2
    difference = difference.where(valid)

    if ax:
        m = ax.pcolormesh(ds['assembled_lon_bounds'],
                          ds['assembled_lat_bounds'],
                          difference,
                          vmin=vmin,
                          vmax=vmax,
                          levels=256,
                          cmap=cmap,
                          cmap_kw={'cut': cut})

        print('min_diff_high', difference.where(valid).min(), 'max_diff_high', difference.where(valid).max())
        return m

    else:
        return amf_lnox, amf_lnox_mod

In [ ]:
## ---------------- AMF_LNOx ---------------- ##
amflnox_min = -50; amflnox_max=50

# --- subplot 4 ---
ax = axs[4]
m = delta_amf_lnox(ds, ds_nolnox, ds_lnox, ds_lnox_high, s5p_p, amflnox_min, amflnox_max, ax=ax, p_max=800)
ax.format(title='Low layer (> 800 hPa)')

# --- lightning scatter ---
s = ax.scatter(ltng['longitude'], ltng['latitude'],
               c=ltng['delta'],
               cmap='plasma_r',
               levels=6,
               s=2)
s.set_facecolor('none')

ax.colorbar(s, loc='l',
            label='Flash Time (mins)',
            shrink=0.87)

# --- subplot 5 ---
ax = axs[5]
delta_amf_lnox(ds, ds_nolnox, ds_lnox, ds_lnox_high, s5p_p, amflnox_min, amflnox_max, ax=ax, p_min=400, p_max=800)
ax.format(title='Middle layer (800 hPa - 400 hPa)')

# --- subplot 6 ---
ax = axs[6]
delta_amf_lnox(ds, ds_nolnox, ds_lnox, ds_lnox_high, s5p_p, amflnox_min, amflnox_max, ax=ax, p_min=150, p_max=400)
ax.format(title='High layer (400 hPa - 150 hPa)')

# --- subplot 7 ---
ax = axs[7]
delta_amf_lnox(ds, ds_nolnox, ds_lnox, ds_lnox_high, s5p_p, amflnox_min, amflnox_max, ax=ax)
ax.format(title='All layers')

# colorbar
ax.colorbar(m, loc='r',
            ticks=plot.arange(-50, 50, 25),
            minorticks=5,
            label='%$\Delta$ AMF$_{LNO_x}$ 700 vs. 500 mol/flash',
            shrink=0.87)

fig

Since the effect of the low layer can be neglected, I don't plot it to decrease the size of figure:

In [ ]:
fig, axs = plot.subplots(proj='pcarree', ncols=3, nrows=2, sharey=3, space=0)
cmap = 'RdBu_r'; cut = 0.1

provinces = load_province()
axs.add_feature(provinces, edgecolor='k', linewidth=.3)

# extend = [118.2, 119.7, 31.5, 32.8]
extend = [118, 119.6, 31.2, 32.8]
lon_d = 0.5; lat_d = 0.5
# amftrop_min = -50; amftrop_max=50
amftrop_min = -100; amftrop_max=100



## ---------------- AMF_Trop ---------------- ##
# --- subplot 0 ---
ax = axs[0]
delta_amf(ds, ds_nolnox, ds_lnox, s5p_p, amftrop_min, amftrop_max, ax=ax, p_min=400, p_max=800, cmap=cmap, cut=cut)
# ax.format(title='Middle layer (800 hPa - 400 hPa)')

# --- subplot 1 ---
ax = axs[1]
delta_amf(ds, ds_nolnox, ds_lnox, s5p_p, amftrop_min, amftrop_max, ax=ax, p_min=150, p_max=400, cmap=cmap, cut=cut)
# ax.format(title='High layer (400 hPa - 150 hPa)')



# --- subplot 2 ---
ax = axs[2]
m = delta_amf(ds, ds_nolnox, ds_lnox, s5p_p, amftrop_min, amftrop_max, ax=ax, cmap=cmap, cut=cut)
# ax.format(title='All layers')

# colorbar
ax.colorbar(m, loc='r',
#             ticks=plot.arange(-50, 50, 25),
#             minorticks=5,
            ticks=plot.arange(-100, 100, 50),
            minorticks=25,
            label='500 vs. 0 mol/flash',
            extend='max',
            shrink=0.9,
           )


## ---------------- AMF_LNOx ---------------- ##
amflnox_min = -30; amflnox_max=30

# --- subplot 3 ---
ax = axs[3]
delta_amf_lnox(ds, ds_nolnox, ds_lnox, ds_lnox_high, s5p_p, amflnox_min, amflnox_max, ax=ax, p_min=400, p_max=800, cmap=cmap, cut=cut)
# ax.format(title='Middle layer (800 hPa - 400 hPa)')

# --- subplot 4 ---
ax = axs[4]
delta_amf_lnox(ds, ds_nolnox, ds_lnox, ds_lnox_high, s5p_p, amflnox_min, amflnox_max, ax=ax, p_min=150, p_max=400, cmap=cmap, cut=cut)
# ax.format(title='High layer (400 hPa - 150 hPa)')

# --- subplot 5 ---
ax = axs[5]
m = delta_amf_lnox(ds, ds_nolnox, ds_lnox, ds_lnox_high, s5p_p, amflnox_min, amflnox_max, ax=ax, cmap=cmap, cut=cut)
# ax.format(title='All layers')

# colorbar
ax.colorbar(m, loc='r',
            ticks=plot.arange(-30, 30, 10),
            minorticks=5,
            label='700 vs. 500 mol/flash',
            extend='min',
            shrink=0.9,
           )

# format
axs[:, 0].format(latlabels=True)
axs[1, :].format(lonlabels=True)

axs.format(abc=False,#True,
#            abcloc='ul',
#            abcstyle='(a)',
           lonlines=lon_d,
           latlines=lat_d,
           lonlim=(extend[0], extend[1]),
           latlim=(extend[2], extend[3]),
           rowlabels=['%$\Delta$ AMF$_{Trop}$ \n (2019-07-25)',
                      '%$\Delta$ AMF$_{LNO_x}$ \n (2019-07-25)'],
           collabels=['Middle Troposphere \n (800 hPa to 400 hPa)',
                      'Upper Troposphere \n (400 hPa to 150 hPa)',
                      'All layers \n (surface to top)'],
           dms=False,
           grid=False,
#            suptitle='2019-07-25',
          )

fig.savefig('../figures/s5p_amf_diff_2019.png')

How about the AMF_trop with 330 mol/flash and 700 mol/flash?

It's hard to check the relationship among CP, CRF and $\Delta$AMF$_{Trop}$.

Let's try to plot the scatter distribution:

In [ ]:
# region_subset = (extend[0] <= ds.coords['longitude'].isel(time=0)) \
#                 & (ds.coords['longitude'].isel(time=0) <= extend[1]) \
#                 & (extend[2] <= ds.coords['latitude'].isel(time=0)) \
#                 & (ds.coords['latitude'].isel(time=0) <= extend[3])
            
# amf, amf_mt = delta_amf(ds, ds_nolnox, ds_lnox, s5p_p, amftrop_min, amftrop_max, p_min=400, p_max=800)
# _, amf_ut = delta_amf(ds, ds_nolnox, ds_lnox, s5p_p, amftrop_min, amftrop_max, p_min=150, p_max=400)
# _, amf_all = delta_amf(ds, ds_nolnox, ds_lnox, s5p_p, amftrop_min, amftrop_max)

# pdiff_mt = ((amf_mt-amf)/amf).where(region_subset)#.stack(dim0=['x', 'y'])
# pdiff_ut = ((amf_ut-amf)/amf).where(region_subset)#.stack(dim0=['x', 'y'])
# pdiff_all = ((amf_all-amf)/amf).where(region_subset)#.stack(dim0=['x', 'y'])

# pdiff_mt_large = pdiff_mt.stack(dim0=['x', 'y'])>0.2
# cp_mt_large = ds['cloud_pressure_crb'].where(region_subset).stack(dim0=['x', 'y']).where(pdiff_mt_large, drop=True)
# crf_mt_large = ds['cloud_fraction_crb_nitrogendioxide_window'].where(region_subset).stack(dim0=['x', 'y']).where(pdiff_mt_large, drop=True)

# pdiff_mt_small = pdiff_mt.stack(dim0=['x', 'y'])<-0.2
# cp_mt_small = ds['cloud_pressure_crb'].where(region_subset).stack(dim0=['x', 'y']).where(pdiff_mt_small, drop=True)
# crf_mt_small = ds['cloud_fraction_crb_nitrogendioxide_window'].where(region_subset).stack(dim0=['x', 'y']).where(pdiff_mt_small, drop=True)

# pdiff_ut_small = pdiff_ut.stack(dim0=['x', 'y'])<-0.2
# cp_ut_small = ds['cloud_pressure_crb'].where(region_subset).stack(dim0=['x', 'y']).where(pdiff_ut_small, drop=True)
# crf_ut_small = ds['cloud_fraction_crb_nitrogendioxide_window'].where(region_subset).stack(dim0=['x', 'y']).where(pdiff_ut_small, drop=True)

# fig, axs = plot.subplots()

# s_mt_large = axs.scatter(crf_mt_large, cp_mt_large, c='red6', label='MT %$\Delta$AMF$_{Trop}$ > 20%')
# s_mt_small = axs.scatter(crf_mt_small, cp_mt_small, c='blue6', label='MT %$\Delta$AMF$_{Trop}$ < -20%')
# s_ut_small = axs.scatter(crf_ut_small, cp_ut_small, c='green6', label='UT %$\Delta$AMF$_{Trop}$ < -5%')


# axs.legend([s_mt_large, s_mt_small, s_ut_small], loc='r', ncols=1)

region_subset = (extend[0] <= ds.coords['longitude'].isel(time=0)) \
                & (ds.coords['longitude'].isel(time=0) <= extend[1]) \
                & (extend[2] <= ds.coords['latitude'].isel(time=0)) \
                & (ds.coords['latitude'].isel(time=0) <= extend[3])
            
amf, amf_mt = delta_amf(ds, ds_nolnox, ds_lnox, s5p_p, amftrop_min, amftrop_max, p_min=400, p_max=800)
_, amf_ut = delta_amf(ds, ds_nolnox, ds_lnox, s5p_p, amftrop_min, amftrop_max, p_min=150, p_max=400)
_, amf_all = delta_amf(ds, ds_nolnox, ds_lnox, s5p_p, amftrop_min, amftrop_max)

pdiff_mt = ((amf_mt-amf)/amf).where(region_subset)#.stack(dim0=['x', 'y'])
pdiff_ut = ((amf_ut-amf)/amf).where(region_subset)#.stack(dim0=['x', 'y'])
pdiff_all = ((amf_all-amf)/amf).where(region_subset)#.stack(dim0=['x', 'y'])

pdiff_ut_large = pdiff_ut.stack(dim0=['x', 'y'])>0.5
cp_ut_large = ds['cloud_pressure_crb'].where(region_subset).stack(dim0=['x', 'y']).where(pdiff_ut_large, drop=True)
crf_ut_large = ds['cloud_fraction_crb_nitrogendioxide_window'].where(region_subset).stack(dim0=['x', 'y']).where(pdiff_ut_large, drop=True)

pdiff_mt_large = pdiff_mt.stack(dim0=['x', 'y'])>0.2
cp_mt_large = ds['cloud_pressure_crb'].where(region_subset).stack(dim0=['x', 'y']).where(pdiff_mt_large, drop=True)
crf_mt_large = ds['cloud_fraction_crb_nitrogendioxide_window'].where(region_subset).stack(dim0=['x', 'y']).where(pdiff_mt_large, drop=True)

pdiff_ut_small = pdiff_ut.stack(dim0=['x', 'y'])<-0.2
cp_ut_small = ds['cloud_pressure_crb'].where(region_subset).stack(dim0=['x', 'y']).where(pdiff_ut_small, drop=True)
crf_ut_small = ds['cloud_fraction_crb_nitrogendioxide_window'].where(region_subset).stack(dim0=['x', 'y']).where(pdiff_ut_small, drop=True)

pdiff_mt_small = pdiff_mt.stack(dim0=['x', 'y'])<-0.2
cp_mt_small = ds['cloud_pressure_crb'].where(region_subset).stack(dim0=['x', 'y']).where(pdiff_mt_small, drop=True)
crf_mt_small = ds['cloud_fraction_crb_nitrogendioxide_window'].where(region_subset).stack(dim0=['x', 'y']).where(pdiff_mt_small, drop=True)

fig, axs = plot.subplots()

s_ut_large = axs.scatter(crf_ut_large, cp_ut_large/1e2, c='violet5', label='UT %$\Delta$AMF$_{Trop}$ > 50%', s=85, zorder=1)
s_mt_large = axs.scatter(crf_mt_large, cp_mt_large/1e2, c='orange5', label='MT %$\Delta$AMF$_{Trop}$ > 20%', s=65, zorder=2)
s_ut_small = axs.scatter(crf_ut_small, cp_ut_small/1e2, c='green5', label='UT %$\Delta$AMF$_{Trop}$ < -5%', s=45, zorder=3)
s_mt_small = axs.scatter(crf_mt_small, cp_mt_small/1e2, c='blue5', label='MT %$\Delta$AMF$_{Trop}$ < -20%', s=25, zorder=7)

axs.legend([s_ut_large, s_mt_large, s_ut_small, s_mt_small], loc='r', ncols=1)
axs.format(grid=False,
           ylabel='Cloud Pressure (hPa)',
           xlabel='f$_{effNO_2}$',
           xlim=(0.3, 1.05),
           ylim=(1000, 100),
           title='2019-07-25',
           abc=True,
           abcloc='ul',
           abcstyle='(a)')

In [ ]:
fig, axs = plot.subplots(proj='pcarree', ncols=2, nrows=2)

provinces = load_province()
axs.add_feature(provinces, edgecolor='k', linewidth=.3)

extend = [118, 119.6, 31.2, 32.8]
lon_d = 0.5; lat_d = 0.5

m0 = axs[0].pcolormesh(ds['assembled_lon_bounds'],
                       ds['assembled_lat_bounds'],
                       pdiff_mt.where(pdiff_mt>0.2)*1e2,
                       vmin=-100,
                       vmax=100,
                       levels=256,
                      cmap='RdBu_r')

axs[0].colorbar([m0])

m1 = axs[1].pcolormesh(ds['assembled_lon_bounds'],
                       ds['assembled_lat_bounds'],
                       pdiff_mt.where(pdiff_mt<-0.2)*1e2,
                       vmin=-100,
                       vmax=100,
                       levels=256,
                      cmap='RdBu_r')

axs[1].colorbar([m1])

m2 = axs[2].pcolormesh(ds['assembled_lon_bounds'],
                       ds['assembled_lat_bounds'],
                       pdiff_ut.where(pdiff_ut<-0.05)*1e2,
                       vmin=-100,
                       vmax=100,
                       levels=256,
                      cmap='RdBu_r')

axs[2].colorbar([m2])

axs.format(abc=True,
           abcloc='l',
           abcstyle='(a)',
           lonlines=lon_d,
           latlines=lat_d,
           lonlim=(extend[0], extend[1]),
           latlim=(extend[2], extend[3]),
           rowlabels=['MT', 'UT'],
#            collabels=['VCD$_{LNO_x}$',
#                       '%$\Delta$VCD$_{LNO_x}$'],
           dms=False,
          )

## VCD$_{LNO_x}$

$ VCD_{LNO_x} = SCD_{NO_2} / AMF_{NO_x} $

In [ ]:
amf_lnox, amf_lnox_mt = delta_amf_lnox(ds, ds_nolnox, ds_lnox, ds_lnox_high, s5p_p, amflnox_min, amflnox_max, p_min=400, p_max=800)

_, amf_lnox_ut = delta_amf_lnox(ds, ds_nolnox, ds_lnox, ds_lnox_high, s5p_p, amflnox_min, amflnox_max, p_min=150, p_max=400)

_, amf_lnox_all = delta_amf_lnox(ds, ds_nolnox, ds_lnox, ds_lnox_high, s5p_p, amflnox_min, amflnox_max)

lnoxTrop = scdTrop/amf_lnox
lnoxTrop_mt = scdTrop/amf_lnox_mt
lnoxTrop_ut = scdTrop/amf_lnox_ut
lnoxTrop_all = scdTrop/amf_lnox_all

In [ ]:
fig, axs = plot.subplots(proj='pcarree')#, ncols=2)

provinces = load_province()
axs.add_feature(provinces, edgecolor='k', linewidth=.3)

extend = [118, 119.6, 31.2, 32.8]
lon_d = 0.5; lat_d = 0.5

# only plot the AMFs related to LNOx (0 < AMF < 1)
valid = (0 < amf_lnox) & (amf_lnox <= 1)
difference = (lnoxTrop_all-lnoxTrop)/lnoxTrop*1e2
difference = difference.where(valid)



cmap='Ice'
cmap_kw = {'left': 0.05, 'right': 0.85}

m0 = axs[0].pcolormesh(ds['assembled_lon_bounds'],
                       ds['assembled_lat_bounds'],
                       lnoxTrop*6.02214e3,
                       vmin=0,
                       vmax=1.5,
                       levels=256,
                       cmap=cmap,
                       cmap_kw=cmap_kw)

axs[0].colorbar([m0], loc='r', ticks=0.25, minorticks=0.125, extend='max', label='VCD$_{LNO_x}$ (10$^{16}$ molec. / cm$^2$)')
axs[0].format(title='')
# axs[0].format(title='VCD$_{LNO_x}$ (2019-07-25)')

ltng_bounds = np.concatenate([np.linspace(begin_t, -30, 6), np.linspace(-20, -0, 3)])
norm = matplotlib.colors.BoundaryNorm(boundaries=ltng_bounds, ncolors=256)

s = axs[0].scatter(ltng['longitude'], ltng['latitude'],
                 c=ltng['delta'],
                 cmap='RdYlGn',
                 cmap_kw={'left':0.1, 'cut': 0.3, 'shift': 180},
                 norm=norm,
                 s=1)

s.set_facecolor("none")
axs[0].colorbar(s, loc='r',
                ticks=ltng_bounds,
                label='Flash Time (mins)')

# m1 = axs[1].pcolormesh(ds['assembled_lon_bounds'],
#                        ds['assembled_lat_bounds'],
#                        difference,
#                        vmin=0,
#                        vmax=120,
#                        levels=256,
#                        cmap=cmap,
#                        cmap_kw=cmap_kw)

# axs[1].colorbar([m1], loc='r', ticks=30, label='(700 vs. 330 mol/flash)')
# axs[1].format(title='%$\Delta$VCD$_{LNO_x}$')


axs.format(
#            abc=True,
#            abcloc='l',
#            abcstyle='(a)',
           lonlines=lon_d,
           latlines=lat_d,
           lonlim=(extend[0], extend[1]),
           latlim=(extend[2], extend[3]),
#            rowlabels=['2019-07-25'],
#            collabels=['VCD$_{LNO_x}$',
#                       '%$\Delta$VCD$_{LNO_x}$'],
           dms=False,
           grid=False,
           lonlabels=True,
           latlabels=True,
          )

fig.savefig('../figures/s5p_vcdlnox_2019.png')

## Comparisons of SCD$_{Trop}$ by replacing profiles

In [ ]:
def cal_scd(ds, ds_lnox, da_mod, s5p_p):
    scdClr = integPr(da_mod * ds_lnox['swClr'],
                     s5p_p,
                     ds['surface_pressure']/1e2, ds_lnox['tropopause_pressure'])

    scdCld = integPr(da_mod * ds_lnox['swCld'],
                     s5p_p,
                     ds['cloud_pressure_crb']/1e2, ds_lnox['tropopause_pressure'])

    crf = ds['cloud_radiance_fraction_nitrogendioxide_window']

    scdTrop = scdClr*(1-crf) + scdCld*crf

    return scdTrop.where(ds['no2_scd_flag']==0)

In [ ]:
scd_nolnox = cal_scd(ds, ds_lnox, ds_nolnox['no2apriori'], s5p_p)
scd_lnox = cal_scd(ds, ds_lnox, ds_lnox['no2apriori'], s5p_p)
scd_lnox_low = cal_scd(ds, ds_lnox, ds_lnox_low['no2apriori'], s5p_p)
scd_lnox_high = cal_scd(ds, ds_lnox, ds_lnox_high['no2apriori'], s5p_p)

In [ ]:
fig, axs = plot.subplots(proj='pcarree', ncols=4)#, space=0)

provinces = load_province()
axs.add_feature(provinces, edgecolor='k', linewidth=.3)

extend = [118, 119.6, 31.2, 32.8]
lon_d = 0.5; lat_d = 0.5

cmap = plot.Colormap('YlGnBu_r', 'YlOrRd', ratios=(1, 1))

m0 = axs[0].pcolormesh(ds['assembled_lon_bounds'],
                       ds['assembled_lat_bounds'],
                       scd_nolnox/1e16,
                       vmin=0,
                       vmax=2,
                       levels=256,
                       cmap=cmap)
axs[0].format(title='0 mol/flash')

m1 = axs[1].pcolormesh(ds['assembled_lon_bounds'],
                       ds['assembled_lat_bounds'],
                       scd_lnox_low/1e16,
                       vmin=0,
                       vmax=2,
                       levels=256,
                       cmap=cmap)
axs[1].format(title='330 mol/flash')

m2 = axs[2].pcolormesh(ds['assembled_lon_bounds'],
                       ds['assembled_lat_bounds'],
                       scd_lnox/1e16,
                       vmin=0,
                       vmax=2,
                       levels=256,
                       cmap=cmap)
axs[2].format(title='500 mol/flash')

m3 = axs[3].pcolormesh(ds['assembled_lon_bounds'],
                       ds['assembled_lat_bounds'],
                       scd_lnox_high/1e16,
                       vmin=0,
                       vmax=2,
                       levels=256,
                       cmap=cmap)
axs[3].format(title='700 mol/flash')

fig.colorbar([m0], loc='r', ticks=0.25, label='10$^{16}$ molec. / cm$^2$', extend='both')

axs.format(abc=False,#True,
#            abcloc='l',
#            abcstyle='(a)',
           lonlines=lon_d,
           latlines=lat_d,
           lonlim=(extend[0], extend[1]),
           latlim=(extend[2], extend[3]),
           rowlabels=['2019-07-25'],
           dms=False,
           lonlabels=True,
           suptitle='a priori SCD$_{tropNO_2}$',
           grid=False,
          )
# format
axs[:, 0].format(latlabels=True)

fig.savefig('../figures/s5p_apriori_scd_2019.png')

## Calculate LNO$_x$ Production